In [6]:
import pandas as pd
import duckdb as dk
import numpy as np

con = dk.connect('///data/vermont.duckdb')

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 2000)

In [33]:
import pandas as pd
personal_raw_df = pd.read_csv(
    "data/MyEBirdData.csv",
    dtype=str,
    keep_default_na=False,
    na_values=[''],
    encoding='utf-8',
    engine='python'
)

print(personal_raw_df.shape)
print(personal_raw_df.head())

(9123, 23)
  Submission ID Common Name     Scientific Name Taxonomic Order Count State/Province      County Location ID                                           Location    Latitude    Longitude        Date      Time                  Protocol Duration (Min) All Obs Reported Distance Traveled (km) Area Covered (ha) Number of Observers Breeding Code                          Observation Details Checklist Comments ML Catalog Numbers
0    S218884326  Snow Goose  Anser caerulescens             267     1          US-VT     Addison   L41289013  2049 Little Chicago Road, Ferrisburgh, Vermont...  44.1991056  -73.2835316  2025-03-16  11:17 AM   eBird - Traveling Count             47                1                   .329               NaN                   1           NaN                                   Blue morph               None               None
1    S284838194  Snow Goose  Anser caerulescens             267  2500          US-VT     Addison     L715236                      Dead Creek WM

In [42]:
# drop hybrids and slashes
personal_raw_df = personal_raw_df[~(
                                  personal_raw_df['Common Name'].str.contains(r'\sx\s', regex=True, case=False) |
                                  personal_raw_df['Common Name'].str.contains(r'\bhybrid', regex=True, case=False) |
                                  personal_raw_df['Common Name'].str.contains(r'\s\\\s', regex=True) |
                                  personal_raw_df['Common Name'].str.contains(r'/', regex=True) |
                                  personal_raw_df['Common Name'].str.contains(r'sp\.', regex=True, case=False)
                                  )]
# remove subspecies classification
personal_raw_df['Common Name'] = personal_raw_df['Common Name'].str.replace(r'\s(?=\().*', '', regex=True)
personal_raw_df.sort_values(by=['Taxonomic Order', 'Date'], inplace=True)
personal_staging_df = personal_raw_df.drop_duplicates(subset=['Common Name'])
personal_clean_df = personal_staging_df[['Common Name','Scientific Name','Taxonomic Order', 'Date']].sort_values(by=['Date'])
personal_clean_df.index = np.arange(1, len(personal_clean_df) + 1)

In [55]:
personal_clean_df_to_duckdb_query = """
DROP TABLE IF EXISTS life_list;
CREATE TABLE life_list AS
SELECT
    "Common Name" as common_name,
    "Scientific Name" as scientific_name,
    "Taxonomic Order" as taxonomic_order,
    TRY_CAST("Date" AS DATE) as date
FROM personal_clean_df;
"""

con.execute(personal_clean_df_to_duckdb_query)

create_unique_index_query = """
CREATE UNIQUE INDEX idx_life_list_scientific_name
ON life_list(scientific_name);
"""

con.execute(create_unique_index_query)